* Tools used in this Project
* Using Pyloader to load PDF files as our dataset
* Using RecursiveTextSplitter to split data 
* Embedding the data by HaggingFaceEmbeddings 
* Using FAISS as vectorstore
* Grok as LLM , Importing it from langchain_grok 
* Making Prompt by ChatPromptTemplate from langchain_core
* Create_Stuf_documents_Chain


In [1]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(r'C:\Users\alibe\Downloads\ML_Chatbot_Guide\ML_For_Beginners.pdf')
data = loader.load()

data

[Document(metadata={'producer': 'calibre 3.31.0 [https://calibre-ebook.com]', 'creator': 'calibre 3.31.0 [https://calibre-ebook.com]', 'creationdate': '2023-01-22T13:46:33+00:00', 'author': 'Theobald, Oliver', 'title': 'Machine Learning for Absolute Beginners: A Plain English Introduction (Third Edition)', 'source': 'C:\\Users\\alibe\\Downloads\\ML_Chatbot_Guide\\ML_For_Beginners.pdf', 'total_pages': 169, 'page': 0, 'page_label': '1'}, page_content=''),
 Document(metadata={'producer': 'calibre 3.31.0 [https://calibre-ebook.com]', 'creator': 'calibre 3.31.0 [https://calibre-ebook.com]', 'creationdate': '2023-01-22T13:46:33+00:00', 'author': 'Theobald, Oliver', 'title': 'Machine Learning for Absolute Beginners: A Plain English Introduction (Third Edition)', 'source': 'C:\\Users\\alibe\\Downloads\\ML_Chatbot_Guide\\ML_For_Beginners.pdf', 'total_pages': 169, 'page': 1, 'page_label': '2'}, page_content=''),
 Document(metadata={'producer': 'calibre 3.31.0 [https://calibre-ebook.com]', 'creat

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1000,
  chunk_overlap=200,
)

docs = text_splitter.split_documents(data)

In [12]:
len(docs)

328

In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS  

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"},
)
vectorstore = FAISS.from_documents(docs , embeddings)

C:\Users\alibe\AppData\Local\Temp\ipykernel_26004\2120828304.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [11]:
vectorstore

In [9]:
retriever = vectorstore.as_retriever(search_type = "similarity",search_kwargs={"k": 3})
retriver_docs = retriever.invoke("What is the difference between supervised and unsupervised learning?")
retriver_docs[0].page_content

'Unsupervised\tLearning\nIn\tthe\tcase\tof\tunsupervised\tlearning,\tthe\toutput\tvariables\tare\tunlabeled,\tand\ncombinations\tof\tinput\tand\toutput\tvariables\taren’t\tknown.\tUnsupervised\tlearning\ninstead\t focuses\t on\t analyzing\t relationships\t between\t input\t variables\t and\nuncovering\thidden\tpatterns\tthat\tcan\tbe\textracted\tto\tcreate\tnew\tlabels\tregarding\npossible\toutputs.\nIf\tyou\tgroup\tdata\tpoints\tbased\ton\tthe\tpurchasing\tbehavior\tof\tSME\t(Small\tand\nMedium-sized\tEnterprises)\tand\tlarge\tenterprise\tcustomers,\tfor\texample,\tyou’re\nlikely\tto\tsee\ttwo\tclusters\tof\tdata\tpoints\temerge.\tThis\tis\tbecause\tSMEs\tand\tlarge\nenterprises\ttend\tto\thave\tdifferent\tprocurement\tneeds.\tWhen\tit\tcomes\tto\npurchasing\tcloud\tcomputing\tinfrastructure,\tfor\texample,\tessential\tcloud\thosting\nproducts\tand\ta\tContent\tDelivery\tNetwork\t(CDN)\tshould\tprove\tsufficient\tfor\nmost\tSME\tcustomers.\tLarge\tenterprise\tcustomers,\tthough,\tare\

In [ ]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv  
load_dotenv()

llm = ChatGroq(
    groq_api_key= Grok_API_KEY, 
    model_name="llama3-70b-8192"
)

In [30]:
from langchain_core.prompts import ChatPromptTemplate
System_prompt = (
    "You are a helpful AI assistant. "
    "You will be given a question and you will answer it based on the provided context. "
    "If you don't know the answer, just say that you don't know. "
    "Do not try to make up an answer."  
    "Use the context provided to answer the question."
    "/n/n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", System_prompt),
        ("human", "{input}"),
    ]
)

In [31]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

question_answer_chain = create_stuff_documents_chain(llm , prompt = prompt)
rag_chain = create_retrieval_chain(retriever , question_answer_chain)

response = rag_chain.invoke({"input": "What is the difference between supervised and unsupervised learning?"})

print(response["answer"])

According to the provided context, the main difference between supervised and unsupervised learning is the presence or absence of labeled output variables.

In supervised learning, the output variables are labeled, and the algorithm learns by comparing known combinations of input and output variables to discern patterns and make predictions.

In unsupervised learning, the output variables are unlabeled, and the algorithm focuses on analyzing relationships between input variables and uncovering hidden patterns to create new labels regarding possible outputs.

In other words, supervised learning has a clear target output to learn from, whereas unsupervised learning does not have a known target output and instead tries to discover patterns and structure in the data.
